In [82]:
!pip install jdc

In [83]:
import pandas as pd
import numpy as np
import jdc

In [84]:
class MDP():
  def __init__(self, num_state, num_actions, filename, gamma):
    self.num_states = num_state
    self.num_actions = num_actions
    self.filename = filename
    self.gamma = gamma

    self.stateDic = {}

    self.read_from_file()

In [170]:
%%add_to MDP
def read_from_file(self):
  file = open(self.filename, "r")

  data_obj = []
  
  for line in file:
    data_obj.append(line)

  file.close()

  for i, obj in enumerate(data_obj):
    obj = obj.rstrip()
    obj = obj.split("(")
    obj = "".join(obj)
    obj = obj.split(")")
    obj = "".join(obj)
    obj = obj.split(" ")
    data_obj[i] = obj

  # print(data_obj)

  for obj in data_obj:
    action = {}
    for a in range(self.num_actions):
      i = a+1
      action['a'+str(i)] = []
    self.stateDic[obj[0]] = {"reward":int(obj[1]), "actions": action}

  # self.stateDic["s1"]["actions"]["a1"].append({"s1": 0.509})
  # self.stateDic["s1"]["actions"]["a1"].append({"s2": 0.491})

  # print(data_obj[2])

  for obj in data_obj:
    idx = 3
    for action in obj[2::3]:
      self.stateDic[obj[0]]["actions"][action].append({obj[idx]: float(obj[idx+1])})
      idx += 3
  
  print(self.stateDic["s2"])

In [171]:
n_state = 3
n_actions = 2
filename = 'test1.in'
discount = 0.5
mdp = MDP(n_state, n_actions, filename, discount)

{'reward': 10, 'actions': {'a1': [{'s1': 0.4}, {'s2': 0.3}, {'s3': 0.3}], 'a2': [{'s2': 0.5}, {'s3': 0.5}]}}
